# Focal cell coordinate finder

In [1]:
import calculate_radial_analysis as calculate
import dataio

import glob
import os
import re
from tqdm.auto import tqdm
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
from natsort import natsorted
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

## 1) Event dictionary loader

Specify cell type to filter out unwanted apoptoses, want end result of event_dict that takes the format of :`event_dict = {event_ID:event_time}` 

where `event_ID` takes the format of: `GV{experiment_ID}_Pos{position_ID}_{cell_ID}_{cell_type}` 

For example: 

```
event_dict = {'GV0794_Pos4_393_GFP': '766',
             'GV0794_Pos7_364_GFP': '550',
             'GV0794_Pos7_457_GFP': '805',
             'GV0794_Pos7_369_GFP': '833',
             'GV0794_Pos7_292_GFP': '884',
             'GV0795_Pos4_358_GFP': '628',
             'GV0795_Pos4_18_GFP': '855',
             'GV0795_Pos4_1356_GFP': '992',
             'GV0795_Pos4_359_GFP': '1133',
             'GV0796_Pos4_542_GFP': '581',
             'GV0796_Pos4_1334_GFP': '719',
             'GV0796_Pos4_1586_GFP': '785'}
 ```
 
Function to apoptosis event dictionary from preexisting data store:

`dataio.apoptosis_list_loader(path_to_apop_lists, cell_type)`

Where the `path_to_apop_lists` is the directory containing the apoptosis information and the `cell_type` is either `'Scr'` or `'WT'`.

In [2]:
path_to_apop_lists = '/home/nathan/data/kraken/scr/h2b/giulia/experiment_information/apoptoses/final/all'
event_dict = dataio.apoptosis_list_loader(path_to_apop_lists, 'Scr')
event_dict

{'GV0797_Pos7_14_RFP': '99',
 'GV0797_Pos7_45_RFP': '201',
 'GV0797_Pos7_5_RFP': '254',
 'GV0797_Pos7_146_RFP': '388',
 'GV0797_Pos7_145_RFP': '398',
 'GV0797_Pos7_30_RFP': '393',
 'GV0797_Pos7_106_RFP': '424',
 'GV0797_Pos7_9_RFP': '405',
 'GV0797_Pos7_204_RFP': '487',
 'GV0797_Pos7_105_RFP': '473',
 'GV0797_Pos7_200_RFP': '486',
 'GV0797_Pos7_199_RFP': '625',
 'GV0797_Pos7_208_RFP': '475',
 'GV0797_Pos7_207_RFP': '475',
 'GV0797_Pos7_203_RFP': '516',
 'GV0797_Pos23_31_RFP': '166',
 'GV0797_Pos23_26_RFP': '60',
 'GV0797_Pos23_260_RFP': '235',
 'GV0797_Pos23_125_RFP': '79',
 'GV0797_Pos23_336_RFP': '456',
 'GV0797_Pos23_252_RFP': '207',
 'GV0797_Pos23_301_RFP': '207',
 'GV0797_Pos23_36_RFP': '180',
 'GV0797_Pos23_319_RFP': '556',
 'GV0797_Pos23_556_RFP': '638',
 'GV0798_Pos22_34_RFP': '54',
 'GV0798_Pos22_36_RFP': '125',
 'GV0798_Pos22_37_RFP': '469',
 'GV0798_Pos22_358_RFP': '461',
 'GV0798_Pos22_26_RFP': '761',
 'GV0798_Pos22_696_RFP': '871',
 'GV0798_Pos22_9_RFP': '190',
 'GV0798_Po

## 2) Load corresponding cell tracking data and specifiy other parameters

`dataio.hdf5_file_finder(parent_folder_hdf5)` loads a list of all hdf5 files when supplied with the parent directory of all the experiments. This relies on a typical sub-parent directory structure of `'GV****/Pos*/*aligned/HDF/segmented.hdf5'` (aligned folder optional)

In [3]:
### load tracking data
parent_folder_hdf5 = '/home/nathan/data/kraken/scr/h2b/giulia'
hdf5_file_list = dataio.hdf5_file_finder(parent_folder_hdf5)


In [ ]:


hdf5_file_path, error_log, success_log = [], [], []
 
for apop_ID in tqdm(event_dict):

    expt = "GV" + str(re.findall(r"GV(\d+)", apop_ID)[0])
    position = re.findall(r"Pos(\d+)", apop_ID)[0]

    position = "Pos" + position

    expt_position = os.path.join(
        expt, position, ""
    )  ## additional '' here so that / added to end of string

    cell_ID = int((re.findall(r"(\d+)_.FP", apop_ID))[0])
    print("ID", apop_ID, "experiment:", expt_position)

    # checks if hdf5 file path already loaded to avoid repeat loading
    if expt_position not in hdf5_file_path:
        ## load that track data
        print("Loading", expt_position)
        hdf5_file_path = [
            hdf5_file_path
            for hdf5_file_path in hdf5_file_list
            if expt_position in hdf5_file_path
        ][0]
        wt_cells, scr_cells, all_cells = dataio.load_tracking_data(hdf5_file_path)
        print("Loaded", expt_position)

    if "RFP" in apop_ID:
        cell_ID = -cell_ID

    focal_time = int(event_dict[apop_ID])

    target_cell = [cell for cell in all_cells if cell.ID == cell_ID][0]

    focal_cell = pd.DataFrame(columns = ['x', 'y', 't', 'ID', 'state'])

    focal_cell['x'], focal_cell['y'],focal_cell['t'], focal_cell['ID'], focal_cell['state'] =target_cell['x'], target_cell['y'],target_cell['t'], target_cell['ID'], target_cell['state']  

    fn = f'/home/nathan/data/kraken/scr/h2b/giulia/experiment_information/apoptoses/apoptotic_tracks/{apop_ID}.csv'

    focal_cell.to_csv(fn)

  0%|          | 0/1839 [00:00<?, ?it/s]

[INFO][2022/03/07 09:36:01 PM] Opening HDF file: /home/nathan/data/kraken/scr/h2b/giulia/GV0797/Pos7/Pos7_aligned/HDF/segmented.hdf5...
[INFO][2022/03/07 09:36:01 PM] Loading tracks/obj_type_1


ID GV0797_Pos7_14_RFP experiment: GV0797/Pos7/
Loading GV0797/Pos7/
Btrack version no.: 0.4.3


[INFO][2022/03/07 09:36:03 PM] Loading objects/obj_type_1 (207794, 5) (196573 filtered: area>=100)
[INFO][2022/03/07 09:36:05 PM] Closing HDF file: /home/nathan/data/kraken/scr/h2b/giulia/GV0797/Pos7/Pos7_aligned/HDF/segmented.hdf5
[INFO][2022/03/07 09:36:05 PM] Opening HDF file: /home/nathan/data/kraken/scr/h2b/giulia/GV0797/Pos7/Pos7_aligned/HDF/segmented.hdf5...
[INFO][2022/03/07 09:36:05 PM] Loading tracks/obj_type_2
[INFO][2022/03/07 09:36:05 PM] Loading objects/obj_type_2 (22939, 5) (18826 filtered: area>=100)
[INFO][2022/03/07 09:36:05 PM] Closing HDF file: /home/nathan/data/kraken/scr/h2b/giulia/GV0797/Pos7/Pos7_aligned/HDF/segmented.hdf5


Track information loaded and ordered according to cell type (WT IDs >0> Scr IDs)
Loaded GV0797/Pos7/
ID GV0797_Pos7_45_RFP experiment: GV0797/Pos7/
ID GV0797_Pos7_5_RFP experiment: GV0797/Pos7/
ID GV0797_Pos7_146_RFP experiment: GV0797/Pos7/
ID GV0797_Pos7_145_RFP experiment: GV0797/Pos7/
ID GV0797_Pos7_30_RFP experiment: GV0797/Pos7/
ID GV0797_Pos7_106_RFP experiment: GV0797/Pos7/
ID GV0797_Pos7_9_RFP experiment: GV0797/Pos7/
ID GV0797_Pos7_204_RFP experiment: GV0797/Pos7/
ID GV0797_Pos7_105_RFP experiment: GV0797/Pos7/
ID GV0797_Pos7_200_RFP experiment: GV0797/Pos7/
ID GV0797_Pos7_199_RFP experiment: GV0797/Pos7/
ID GV0797_Pos7_208_RFP experiment: GV0797/Pos7/


...

[INFO][2022/03/07 09:40:34 PM] Loading objects/obj_type_1 (191590, 5) (179288 filtered: area>=100)
[INFO][2022/03/07 09:40:37 PM] Closing HDF file: /home/nathan/data/kraken/scr/h2b/giulia/GV0820/Pos3/Pos3_aligned/HDF/segmented.hdf5
[INFO][2022/03/07 09:40:37 PM] Opening HDF file: /home/nathan/data/kraken/scr/h2b/giulia/GV0820/Pos3/Pos3_aligned/HDF/segmented.hdf5...
[INFO][2022/03/07 09:40:37 PM] Loading tracks/obj_type_2
[INFO][2022/03/07 09:40:37 PM] Loading objects/obj_type_2 (65619, 5) (55005 filtered: area>=100)
[INFO][2022/03/07 09:40:38 PM] Closing HDF file: /home/nathan/data/kraken/scr/h2b/giulia/GV0820/Pos3/Pos3_aligned/HDF/segmented.hdf5
[INFO][2022/03/07 09:40:38 PM] Opening HDF file: /home/nathan/data/kraken/scr/h2b/giulia/GV0798/Pos20/Pos20_aligned/HDF/segmented.hdf5...


Track information loaded and ordered according to cell type (WT IDs >0> Scr IDs)
Loaded GV0820/Pos3/
ID GV0820_Pos3_11_RFP experiment: GV0820/Pos3/
ID GV0820_Pos3_8_RFP experiment: GV0820/Pos3/
ID GV0820_Pos3_21_RFP experiment: GV0820/Pos3/
ID GV0820_Pos3_376_RFP experiment: GV0820/Pos3/
ID GV0820_Pos3_377_RFP experiment: GV0820/Pos3/
ID GV0820_Pos3_10_RFP experiment: GV0820/Pos3/
ID GV0820_Pos3_509_RFP experiment: GV0820/Pos3/
ID GV0820_Pos3_545_RFP experiment: GV0820/Pos3/
ID GV0820_Pos3_508_RFP experiment: GV0820/Pos3/
ID GV0820_Pos3_400_RFP experiment: GV0820/Pos3/
ID GV0820_Pos3_536_RFP experiment: GV0820/Pos3/
ID GV0820_Pos3_535_RFP experiment: GV0820/Pos3/
ID GV0820_Pos3_441_RFP experiment: GV0820/Pos3/
ID GV0820_Pos3_544_RFP experiment: GV0820/Pos3/
ID GV0820_Pos3_442_RFP experiment: GV0820/Pos3/
ID GV0820_Pos3_747_RFP experiment: GV0820/Pos3/
ID GV0820_Pos3_746_RFP experiment: GV0820/Pos3/
ID GV0820_Pos3_38_RFP experiment: GV0820/Pos3/
ID GV0820_Pos3_6_RFP experiment: GV0820/P

[INFO][2022/03/07 09:40:38 PM] Loading tracks/obj_type_1
[INFO][2022/03/07 09:40:39 PM] Loading objects/obj_type_1 (192967, 5) (184679 filtered: area>=100)
[INFO][2022/03/07 09:40:43 PM] Closing HDF file: /home/nathan/data/kraken/scr/h2b/giulia/GV0798/Pos20/Pos20_aligned/HDF/segmented.hdf5
[INFO][2022/03/07 09:40:43 PM] Opening HDF file: /home/nathan/data/kraken/scr/h2b/giulia/GV0798/Pos20/Pos20_aligned/HDF/segmented.hdf5...
[INFO][2022/03/07 09:40:43 PM] Loading tracks/obj_type_2
[INFO][2022/03/07 09:40:43 PM] Loading objects/obj_type_2 (51909, 5) (44768 filtered: area>=100)
[INFO][2022/03/07 09:40:43 PM] Closing HDF file: /home/nathan/data/kraken/scr/h2b/giulia/GV0798/Pos20/Pos20_aligned/HDF/segmented.hdf5
[INFO][2022/03/07 09:40:44 PM] Opening HDF file: /home/nathan/data/kraken/scr/h2b/giulia/GV0798/Pos21/Pos21_aligned/HDF/segmented.hdf5...
[INFO][2022/03/07 09:40:44 PM] Loading tracks/obj_type_1


Track information loaded and ordered according to cell type (WT IDs >0> Scr IDs)
Loaded GV0798/Pos20/
ID GV0798_Pos20_246_RFP experiment: GV0798/Pos20/
ID GV0798_Pos20_267_RFP experiment: GV0798/Pos20/
ID GV0798_Pos20_311_RFP experiment: GV0798/Pos20/
ID GV0798_Pos20_263_RFP experiment: GV0798/Pos20/
ID GV0798_Pos20_240_RFP experiment: GV0798/Pos20/
ID GV0798_Pos20_173_RFP experiment: GV0798/Pos20/
ID GV0798_Pos20_310_RFP experiment: GV0798/Pos20/
ID GV0798_Pos20_343_RFP experiment: GV0798/Pos20/
ID GV0798_Pos20_442_RFP experiment: GV0798/Pos20/
ID GV0798_Pos20_502_RFP experiment: GV0798/Pos20/
ID GV0798_Pos20_503_RFP experiment: GV0798/Pos20/
ID GV0798_Pos20_514_RFP experiment: GV0798/Pos20/
ID GV0798_Pos20_513_RFP experiment: GV0798/Pos20/
ID GV0798_Pos20_441_RFP experiment: GV0798/Pos20/
ID GV0798_Pos20_190_RFP experiment: GV0798/Pos20/
ID GV0798_Pos20_581_RFP experiment: GV0798/Pos20/
ID GV0798_Pos20_580_RFP experiment: GV0798/Pos20/
ID GV0798_Pos21_28_RFP experiment: GV0798/Pos21/

[INFO][2022/03/07 09:40:46 PM] Loading objects/obj_type_1 (332657, 5) (316064 filtered: area>=100)
[INFO][2022/03/07 09:40:50 PM] Closing HDF file: /home/nathan/data/kraken/scr/h2b/giulia/GV0798/Pos21/Pos21_aligned/HDF/segmented.hdf5
[INFO][2022/03/07 09:40:50 PM] Opening HDF file: /home/nathan/data/kraken/scr/h2b/giulia/GV0798/Pos21/Pos21_aligned/HDF/segmented.hdf5...
[INFO][2022/03/07 09:40:50 PM] Loading tracks/obj_type_2
[INFO][2022/03/07 09:40:50 PM] Loading objects/obj_type_2 (59184, 5) (40538 filtered: area>=100)
[INFO][2022/03/07 09:40:52 PM] Closing HDF file: /home/nathan/data/kraken/scr/h2b/giulia/GV0798/Pos21/Pos21_aligned/HDF/segmented.hdf5


In [ ]:
### finding all scr apoptoses in uninduced datasets?